In [2]:
import sympy
from sympy import symbol,symbols, Product, Sum, Array, N
from sympy import expr
from scipy.stats import binom_test
import numpy as np
import pandas as pd

### 7 regions

```python
PFC      229  
NAc      316  
LS      1734  
BNST     424  
LH       300  
BA       171  
CeA       46  

229+316+1734+424+300+171+46 = 3220 total projections
Obs cells: 2494
```

In [9]:
N0,k = symbols('N_0,k')

In [10]:
s = Array([229,316,1734,424,300,171,46])

Need to find the total number of cells included unobserved cells that did not project to any of the 7 sampled brain regions, which we denote $N0$. We can compute this by solving a polynomial based on a binomial distribution of projections. We used the observed frequencies of projections to each brain region to define projection probabilities to each region.

In [11]:
m = 6
pi = (1 - Product((1 - (s[k]/N0)),(k, 0, m)).doit())
soln = sympy.solve(pi * N0 - 2494)
roots = [N(x) for x in soln]
roots

[45.9769119422404,
 4211.84735324631,
 122.696457058644 - 41.6797347313107*I,
 122.696457058644 + 41.6797347313107*I,
 127.252980595015 - 192.326443046457*I,
 127.252980595015 + 192.326443046457*I]

Since $N_0$ is a count, the complex solutions are invalid, and the solution of 45.97... is invalid since it is lower than the observed number of cells. That leaves the true solution of 4211 as the number of total cells.

In [12]:
sympy.simplify((1 - Product((1 - (s[k]/N0)),(k, 0, m)))).doit()

1 - (N_0 - 1734)*(N_0 - 424)*(N_0 - 316)*(N_0 - 300)*(N_0 - 229)*(N_0 - 171)*(N_0 - 46)/N_0**7

No we can compute the probabilities of (independently) projecting to each region based on the empirical data. We do not use these probabilities, however, when constructing our null model.

In [14]:
ps = [(x/4212.0) for x in np.array([229,316,1734,424,300,171,46])]
ps

[0.0543684710351377,
 0.07502374169040836,
 0.4116809116809117,
 0.100664767331434,
 0.07122507122507123,
 0.0405982905982906,
 0.010921177587844255]

In [15]:
psdict = {
'PFC' : 0.0543684710351377,
'NAc' : 0.07502374169040836,
'LS'  : 0.4116809116809117,
'BNST' : 0.100664767331434,  
'LH' : 0.07122507122507123,  
'BA' : 0.0405982905982906,  
'CeA' : 0.010921177587844255  
}

We want our null model to assume conditional independence and no region preference, however, we do want the random graph model to have the same edge density as our empirical data. Using this assumption, we can solve another polynomial equation to find $p_e$, the probability of edge formation for our random graph null model. 

In [20]:
pe = symbols('p_e')

In [22]:
sympy.solve((1-(1-pe)**7) * 4211.84735324631 - 2494,[pe],force=True)

[0.120250862788741,
 0.45148538475476 - 0.687815572145849*I,
 0.45148538475476 + 0.687815572145849*I,
 1.19576259965951 - 0.857691989585134*I,
 1.19576259965951 + 0.857691989585134*I,
 1.79262658419136 - 0.381708845138667*I,
 1.79262658419136 + 0.381708845138667*I]

The only real solution to the polynomial is 0.120250862788741, thus this is our $p_e$. Uing this as our $p_e$ , we can compute the expected number of neurons with at least 1 projection and confirm that it equals our empirical value.

In [25]:
pe_num = 0.120287583118229
(1-(1-pe_num)**7) * 4211

2494.0000002810016

We can also compute the probability of every motif type. For example, the following is the probability of a double projector.

In [27]:
(pe_num)**2 * (1-pe_num)**5

0.007623338558498366

And we can compute the expected number of this type of motif.

In [28]:
0.007623338558498366 * 4211

32.101878669836616

We can also compute the variance and standard deviation of each motif, since the variance of the binomial distribution is $Var = np(1-p)$ and $SD = \sqrt{np(1-p)}$.

In [29]:
np.sqrt(0.007623338558498366 * 4211 * (1-0.007623338558498366))

5.644214310280272

Finally, we can compute the one or two-sided p-value for a particular motif using the binomial test.

In [30]:
binom_test(94,n=4211,p=0.007623338558498366)

4.124134350276005e-19